In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.9
    Uninstalling openai-1.59.9:
      Successfully uninstalled openai-1.59.9


In [2]:
!pip install ffmpeg-python pydub librosa noisereduce soundfile
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

### Tratamento do audio

In [4]:
from google.colab import files
import os

def upload_file():
    """
    Permite o upload de arquivos e salva localmente, verificando o tipo do arquivo.
    """
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Arquivo recebido: {filename}")
        file_ext = filename.split('.')[-1].lower()
        if file_ext == "mp4":
            return filename, 'video'
        elif file_ext == "mp3":
            return filename, 'mp3'
        elif file_ext == "wav":
            return filename, 'wav'
        else:
            print("Tipo de arquivo não suportado.")
            return None, None
    return None, None

print("### Upload do arquivo ###")
file_name, file_type = upload_file()

### Upload do arquivo ###


Saving video_teste_medio.mp4 to video_teste_medio.mp4
Arquivo recebido: video_teste_medio.mp4


In [6]:
if not file_name:0
  print("Nenhum arquivo válido foi carregado.")

IndentationError: unexpected indent (<ipython-input-6-cabd4bb9f90f>, line 2)

* Extração do Áudio e Verificação do Tamanho

In [8]:
import ffmpeg
import os
from pydub import AudioSegment

def check_file_size(audio_path, max_size_mb=25):
    """
    Verifica o tamanho do arquivo em MB.
    """
    file_size_mb = os.path.getsize(audio_path) / (1024 * 1024)  # Convertendo para MB
    print(f"Tamanho do arquivo: {file_size_mb:.2f} MB")
    return file_size_mb <= max_size_mb

Processamento video

In [9]:
import ffmpeg
from pydub import AudioSegment

def extract_and_split_audio_from_video(video_path, output_folder="audio_segments", max_segment_size_mb=24):
    """
    Extrai o áudio do vídeo e divide em segmentos menores que 24 MB.
    """
    print("Extraindo áudio do vídeo...")
    audio_output_path = "full_audio.wav"
    ffmpeg.input(video_path).output(audio_output_path, format='wav', acodec='pcm_s16le', ar='16000').run(quiet=True, overwrite_output=True)
    print(f"Áudio extraído: {audio_output_path}")

    # Verificar e dividir o áudio
    return split_audio_into_segments(audio_output_path, output_folder, max_segment_size_mb)

def split_audio_into_segments(audio_path, output_folder, max_segment_size_mb):
    """
    Divide o áudio extraído em segmentos de até 24MB.
    """
    print(f"Dividindo áudio: {audio_path}")
    audio = AudioSegment.from_wav(audio_path)
    os.makedirs(output_folder, exist_ok=True)

    # Calcular a duração do segmento baseado no tamanho máximo
    bytes_per_second = len(audio.raw_data) / (len(audio) / 1000)
    max_duration_ms = (max_segment_size_mb * 1024 * 1024) / bytes_per_second * 1000

    segments = []
    for i, start_ms in enumerate(range(0, len(audio), int(max_duration_ms))):
        segment = audio[start_ms:start_ms + int(max_duration_ms)]
        segment_path = os.path.join(output_folder, f"segment_{i + 1}.wav")
        segment.export(segment_path, format="wav")
        segments.append(segment_path)
        print(f"Segmento criado: {segment_path} (Tamanho: {os.path.getsize(segment_path) / (1024 * 1024):.2f} MB)")

    return segments

Processamento mp3

In [10]:
import librosa
import noisereduce as nr
import soundfile as sf

def process_audio_mp3(mp3_path, output_dir):
    """
    Processa o áudio MP3, reduzindo ruído e salvando o arquivo limpo.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Reduzindo ruído: {mp3_path}")
    audio_data, sr = librosa.load(mp3_path, sr=None)
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sr)

    output_audio_path = os.path.join(output_dir, f"cleaned_{os.path.basename(mp3_path)}")
    sf.write(output_audio_path, reduced_noise, sr)
    print(f"Áudio limpo salvo em: {output_audio_path}")
    return output_audio_path

Processamento de WAV (Segmentação direta)

In [11]:
def process_audio_wav(wav_path, output_folder):
    """
    Processa o áudio WAV diretamente dividindo-o em segmentos menores de 24 MB.
    """
    print(f"Dividindo WAV diretamente: {wav_path}")
    return split_audio_into_segments(wav_path, output_folder, 24)

In [12]:
if file_type == "video":
    print("Processando arquivo de vídeo...")
    segments = extract_and_split_audio_from_video(file_name)
elif file_type == "mp3":
    print("Processando arquivo MP3...")
    output_dir = "./cleaned_audio_mp3"
    cleaned_audio_path = process_audio_mp3(file_name, output_dir)
    segments = [cleaned_audio_path]  # Como é um arquivo único, não precisa dividir
elif file_type == "wav":
    print("Processando arquivo WAV...")
    output_folder = "./audio_segments_wav"
    segments = process_audio_wav(file_name, output_folder)

print(f"Total de arquivos a serem processados: {len(segments)}")

Processando arquivo de vídeo...
Extraindo áudio do vídeo...
Áudio extraído: full_audio.wav
Dividindo áudio: full_audio.wav
Segmento criado: audio_segments/segment_1.wav (Tamanho: 24.00 MB)
Segmento criado: audio_segments/segment_2.wav (Tamanho: 24.00 MB)
Segmento criado: audio_segments/segment_3.wav (Tamanho: 24.00 MB)
Segmento criado: audio_segments/segment_4.wav (Tamanho: 24.00 MB)
Segmento criado: audio_segments/segment_5.wav (Tamanho: 24.00 MB)
Segmento criado: audio_segments/segment_6.wav (Tamanho: 23.54 MB)
Total de arquivos a serem processados: 6


 ### Redução de Ruído e Transcrição

In [13]:
import librosa
import noisereduce as nr
import soundfile as sf

'''
def process_audio(audio_path, output_audio_path):
    """
    Reduz ruído no áudio e salva um novo arquivo.
    """
    print(f"Reduzindo ruído: {audio_path}")
    audio_data, sr = librosa.load(audio_path, sr=None)
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sr)
    sf.write(output_audio_path, reduced_noise, sr)
    return output_audio_path
'''

def process_audio(audio_path, output_dir):
    """
    Reduz ruído no áudio e salva em um diretório específico.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Reduzindo ruído: {audio_path}")
    audio_data, sr = librosa.load(audio_path, sr=None)
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sr)

    output_audio_path = os.path.join(output_dir, f"cleaned_{os.path.basename(audio_path)}")
    sf.write(output_audio_path, reduced_noise, sr)
    print(f"Áudio limpo salvo em: {output_audio_path}")
    return output_audio_path

In [14]:
'''
def transcribe_audio(audio_path):
    """
    Transcreve o áudio usando a API Whisper da OpenAI.
    """
    print(f"Transcrevendo: {audio_path}")
    with open(audio_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file, language="pt")
    return transcript['text']

'''

def transcribe_audio(audio_path):
    """
    Transcreve o áudio usando a API Whisper da OpenAI.
    """
    print(f"Transcrevendo: {audio_path}")
    with open(audio_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file, language="pt")
    return transcript['text']

In [15]:
cleaned_audio_dir = "./cleaned_audio"  # Pasta onde os áudios limpos serão salvos
if not os.path.exists(cleaned_audio_dir):
    os.makedirs(cleaned_audio_dir)

full_transcription = ""
for idx, segment_path in enumerate(segments):
    print(f"Processando segmento {idx+1}/{len(segments)}...")
    cleaned_audio_path = process_audio(segment_path, cleaned_audio_dir)
    transcription = transcribe_audio(cleaned_audio_path)
    full_transcription += transcription + "\n"

with open("full_transcription.txt", "w", encoding="utf-8") as f:
    f.write(full_transcription)

print("Transcrição concluída e salva em 'full_transcription.txt'.")

Processando segmento 1/6...
Reduzindo ruído: audio_segments/segment_1.wav
Áudio limpo salvo em: ./cleaned_audio/cleaned_segment_1.wav
Transcrevendo: ./cleaned_audio/cleaned_segment_1.wav
Processando segmento 2/6...
Reduzindo ruído: audio_segments/segment_2.wav
Áudio limpo salvo em: ./cleaned_audio/cleaned_segment_2.wav
Transcrevendo: ./cleaned_audio/cleaned_segment_2.wav
Processando segmento 3/6...
Reduzindo ruído: audio_segments/segment_3.wav
Áudio limpo salvo em: ./cleaned_audio/cleaned_segment_3.wav
Transcrevendo: ./cleaned_audio/cleaned_segment_3.wav
Processando segmento 4/6...
Reduzindo ruído: audio_segments/segment_4.wav
Áudio limpo salvo em: ./cleaned_audio/cleaned_segment_4.wav
Transcrevendo: ./cleaned_audio/cleaned_segment_4.wav
Processando segmento 5/6...
Reduzindo ruído: audio_segments/segment_5.wav
Áudio limpo salvo em: ./cleaned_audio/cleaned_segment_5.wav
Transcrevendo: ./cleaned_audio/cleaned_segment_5.wav
Processando segmento 6/6...
Reduzindo ruído: audio_segments/segme

In [16]:
'''
# Processar e transcrever os segmentos
full_transcription = ""
for idx, segment_path in enumerate(segments):
    print(f"Processando segmento {idx+1}/{len(segments)}...")
    cleaned_audio = process_audio(segment_path, f"cleaned_{os.path.basename(segment_path)}")
    transcription = transcribe_audio(cleaned_audio)
    full_transcription += transcription + "\n"

with open("full_transcription.txt", "w", encoding="utf-8") as f:
    f.write(full_transcription)

print("Transcrição concluída e salva em 'full_transcription.txt'.")
'''

'\n# Processar e transcrever os segmentos\nfull_transcription = ""\nfor idx, segment_path in enumerate(segments):\n    print(f"Processando segmento {idx+1}/{len(segments)}...")\n    cleaned_audio = process_audio(segment_path, f"cleaned_{os.path.basename(segment_path)}")\n    transcription = transcribe_audio(cleaned_audio)\n    full_transcription += transcription + "\n"\n\nwith open("full_transcription.txt", "w", encoding="utf-8") as f:\n    f.write(full_transcription)\n\nprint("Transcrição concluída e salva em \'full_transcription.txt\'.")\n'

### Geração das Atas da Reunião

In [17]:
import math

def split_text_by_length(text, max_char_length=8000):
    """
    Divide o texto em partes menores para garantir que cada parte não ultrapasse o limite da API.
    :param text: Texto completo que precisa ser dividido.
    :param max_char_length: Limite de caracteres para cada parte (aproximado para tokens).
    :return: Lista com as partes do texto.
    """
    print("Texto muito longo. Dividindo em partes...")
    parts = []
    while len(text) > max_char_length:
        split_index = text[:max_char_length].rfind('.')  # Encontrar o último ponto para dividir de forma natural
        if split_index == -1:
            split_index = max_char_length  # Divisão forçada se não encontrar ponto
        parts.append(text[:split_index + 1])
        text = text[split_index + 1:].strip()
    parts.append(text)
    print(f"Texto dividido em {len(parts)} partes.")
    return parts

In [18]:
def summarize_text_as_minutes(text, part_number=1):
    """
    Gera uma ata de reunião no formato especificado usando GPT-4.
    """
    prompt = f"""
    Você é um assistente especializado em gerar atas de reuniões. A partir da transcrição a seguir, gere uma ata no formato:

    1. Resumo Geral: Breve descrição do propósito e resultado da reunião.
    2. Principais Tópicos: Liste os principais assuntos discutidos.
    3. Ações e Responsáveis: Identifique ações importantes e quem são os responsáveis.
    4. Métricas e Decisões: Liste números, decisões e acordos feitos.

    A ata deve estar na norma culta da língua portuguesa e devidamente bem detalhada.

    Transcrição (Parte {part_number}):
    {text}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente especialista em atas de reuniões."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    return response['choices'][0]['message']['content']

In [19]:
'''
# Gerar ata da reunião
print("Gerando a ata da reunião...")
final_summary = summarize_text_as_minutes(full_transcription)
print("\n### Ata Final da Reunião ###")
print(final_summary)

# Salvar ata
with open("meeting_minutes.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)
'''

'\n# Gerar ata da reunião\nprint("Gerando a ata da reunião...")\nfinal_summary = summarize_text_as_minutes(full_transcription)\nprint("\n### Ata Final da Reunião ###")\nprint(final_summary)\n\n# Salvar ata\nwith open("meeting_minutes.txt", "w", encoding="utf-8") as f:\n    f.write(final_summary)\n'

In [20]:
atas_dir = "./atas"
if not os.path.exists(atas_dir):
    os.makedirs(atas_dir)

# Simulação de uma transcrição muito longa
print("Verificando o tamanho da transcrição...")
max_char_limit = 8000

# Dividir a transcrição se necessário
if len(full_transcription) > max_char_limit:
    transcription_parts = split_text_by_length(full_transcription, max_char_limit)
else:
    transcription_parts = [full_transcription]

final_summaries = []
for i, part in enumerate(transcription_parts):
    print(f"Gerando a ata da Parte {i+1}/{len(transcription_parts)}...")
    summary = summarize_text_as_minutes(part, part_number=i+1)
    final_summaries.append(summary)
    print(f"Ata da Parte {i+1} concluída.\n")

# Salvar as atas na pasta 'atas'
for i, summary in enumerate(final_summaries):
    file_name = os.path.join(atas_dir, f"ata_parte_{i+1}.txt")
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(summary)
    print(f"Ata Parte {i+1} salva em '{file_name}'.")

# Exibição das atas finais
print("\n### Atas Finais da Reunião ###")
for i, summary in enumerate(final_summaries):
    print(f"\n### Ata Parte {i+1} ###\n")
    print(summary)

Verificando o tamanho da transcrição...
Texto muito longo. Dividindo em partes...
Texto dividido em 5 partes.
Gerando a ata da Parte 1/5...
Ata da Parte 1 concluída.

Gerando a ata da Parte 2/5...
Ata da Parte 2 concluída.

Gerando a ata da Parte 3/5...
Ata da Parte 3 concluída.

Gerando a ata da Parte 4/5...
Ata da Parte 4 concluída.

Gerando a ata da Parte 5/5...
Ata da Parte 5 concluída.

Ata Parte 1 salva em './atas/ata_parte_1.txt'.
Ata Parte 2 salva em './atas/ata_parte_2.txt'.
Ata Parte 3 salva em './atas/ata_parte_3.txt'.
Ata Parte 4 salva em './atas/ata_parte_4.txt'.
Ata Parte 5 salva em './atas/ata_parte_5.txt'.

### Atas Finais da Reunião ###

### Ata Parte 1 ###

1. Resumo Geral: 
A reunião, realizada em formato de podcast, discutiu principalmente os testes de pós-temporada do campeonato de MotoGP de 2024 e as previsões para a temporada de 2025. O encontro contou com a presença de Carlos Costa, Guilherme Longo e Fausto Maceira, comentarista do Mundial de Moto Velocidade na 

### Agregando todas as atas e criando um reusmo geral

In [21]:
import os
import glob
import openai

def read_meeting_parts_from_directory(directory, file_pattern="*.txt"):
    """
    Lê automaticamente todos os arquivos de texto de um diretório e retorna o conteúdo como uma lista.
    :param directory: Caminho do diretório onde os arquivos estão localizados.
    :param file_pattern: Padrão dos arquivos a serem lidos (default é *.txt).
    :return: Lista com o conteúdo de cada arquivo.
    """
    file_paths = glob.glob(os.path.join(directory, file_pattern))
    meeting_parts = []

    print(f"Encontrados {len(file_paths)} arquivos no diretório '{directory}'.")

    for file_path in sorted(file_paths):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            meeting_parts.append(content)
        print(f"Arquivo '{file_path}' lido com sucesso.")

    return meeting_parts

def combine_meeting_parts(meeting_parts):
    """
    Une todas as partes da ata em uma ata final coesa.
    """
    combined_text = "\n".join(meeting_parts)
    print("Todas as partes foram combinadas em uma única ata.")
    return combined_text

In [22]:
def generate_full_summary(meeting_parts):
    """
    Gera um resumo extenso e detalhado de todas as atas em um único parágrafo.
    """
    prompt = f"""
    Abaixo estão várias atas de uma reunião. Gere um resumo total, extenso e detalhado, contendo o máximo de informações possíveis,
    porém em um único parágrafo. O objetivo é criar um texto longo que resuma com precisão tudo que foi discutido.

    Atas:
    {meeting_parts}

    Resumo Extenso e Detalhado:
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente especialista em criar resumos extensos e detalhados de atas."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    return response['choices'][0]['message']['content']

In [23]:
def generate_aggregated_minutes(meeting_parts):
    """
    Agrega todas as atas em uma só ata com um formato estruturado.
    """
    prompt = f"""
    Abaixo estão várias atas de uma reunião. Agrege todas em uma única ata, escolhendo as principais informações de cada uma.
    Use a norma culta da língua portuguesa e siga o formato abaixo:

    1. Resumo Geral: Breve descrição do propósito e resultado da reunião.
    2. Principais Tópicos: Liste os principais assuntos discutidos.
    3. Ações e Responsáveis: Identifique ações importantes e quem são os responsáveis.
    4. Métricas e Decisões: Liste números, decisões e acordos feitos.

    Atas:
    {meeting_parts}

    Ata Consolidada:
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente especialista em criar atas consolidadas e detalhadas."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )
    return response['choices'][0]['message']['content']

In [24]:
directory_path = "./atas"

print("Lendo arquivos do diretório...")
meeting_parts = read_meeting_parts_from_directory(directory_path)

print("Consolidando a ata final...")
final_ata = combine_meeting_parts(meeting_parts)
print("Gerando resumo extenso e detalhado...")
full_summary = generate_full_summary("\n".join(meeting_parts))
print("Gerando ata consolidada...")
aggregated_minutes = generate_aggregated_minutes("\n".join(meeting_parts))

with open("ata_final_completa.txt", "w", encoding="utf-8") as f:
    f.write(final_ata)
print("Ata final salva em 'ata_final_completa.txt'.")

with open("resumo_extenso.txt", "w", encoding="utf-8") as f:
    f.write(full_summary)
print("Resumo extenso salvo em 'resumo_extenso.txt'.")

with open("ata_consolidada.txt", "w", encoding="utf-8") as f:
    f.write(aggregated_minutes)
print("Ata consolidada salva em 'ata_consolidada.txt'.")

Lendo arquivos do diretório...
Encontrados 5 arquivos no diretório './atas'.
Arquivo './atas/ata_parte_1.txt' lido com sucesso.
Arquivo './atas/ata_parte_2.txt' lido com sucesso.
Arquivo './atas/ata_parte_3.txt' lido com sucesso.
Arquivo './atas/ata_parte_4.txt' lido com sucesso.
Arquivo './atas/ata_parte_5.txt' lido com sucesso.
Consolidando a ata final...
Todas as partes foram combinadas em uma única ata.
Gerando resumo extenso e detalhado...
Gerando ata consolidada...
Ata final salva em 'ata_final_completa.txt'.
Resumo extenso salvo em 'resumo_extenso.txt'.
Ata consolidada salva em 'ata_consolidada.txt'.


In [25]:
print("\n### Resumo Extenso e Detalhado ###\n")
print(full_summary)


### Resumo Extenso e Detalhado ###

A reunião, realizada como um podcast, focou principalmente na avaliação do campeonato MotoGP de 2024 e nas previsões para a temporada de 2025. Os participantes, incluindo Carlos Costa, Guilherme Longo e Fausto Maceira, discutiram o título de Jorge Martins, as transições de pilotos e equipes, os testes preliminares e as expectativas para a próxima temporada. Avaliações dos testes realizados em Barcelona e o desempenho de pilotos como Alex Marques, Fábio Quartararo, Peco Banhaia e Jorge Martins foram discutidos, além de questões de patrocínio e marketing. 

Os participantes também discutiram o desempenho de pilotos como Martim, Zarco, Rins, Quartararo, Raul Fernandes, Zarco, Mir, Binder, Acosta, Vachellini, Miguel Oliveira, Miller, Sonqueat Champa, Altaguer, Bagnani, Marchi e Jorge Martini, bem como o desempenho das equipes Yamaha, Honda, Aprilia, KTM, Suzuki e Ducati. A possibilidade de retorno da Suzuki à MotoGP, questões de financiamento e desenvol

In [26]:
print("\n### Ata Consolidada ###\n")
print(aggregated_minutes)


### Ata Consolidada ###

1. Resumo Geral: 
A reunião foi uma discussão abrangente sobre o mundo do motociclismo, cobrindo o desempenho dos pilotos e equipes em 2024 e as previsões para 2025 na MotoGP, assim como os resultados recentes de outras competições de motociclismo, incluindo a Moto2, Moto3, Supercross e a Copa do Mundo de Corridas na Areia. Os participantes expressaram suas opiniões sobre os pilotos, análises das performances das equipes e estratégias para a próxima temporada, além de discutir sobre os desafios e particularidades de corridas na areia.

2. Principais Tópicos:
- Avaliação do desempenho de pilotos na MotoGP, Moto2 e Moto3, incluindo Banhia, Marques, Martim, Acosta, Davi Alonso, Colin Weier, Tony Arbolino, Isa Guevara, Ivan Rostolá, e outros.
- Análise do desempenho das equipes na MotoGP, incluindo Yamaha, Honda, Aprilha, KTM, Suzuki, Ducati, e suas respectivas estratégias.
- Discussão sobre o retorno potencial da Suzuki à MotoGP.
- Avaliação dos testes de pré-tem

In [27]:
import importlib.metadata

# Lista das bibliotecas
bibliotecas = [
    "os",          # Biblioteca padrão do Python, não precisa de instalação
    "openai",
    "ffmpeg",
    "librosa",
    "noisereduce",
    "soundfile",
    "pydub",
    "glob"         # Biblioteca padrão do Python, não precisa de instalação
]

# Verifica a versão de cada biblioteca
for lib in bibliotecas:
    try:
        versao = importlib.metadata.version(lib)
        print(f"{lib}: {versao}")
    except importlib.metadata.PackageNotFoundError:
        print(f"{lib}: Biblioteca padrão do Python ou não instalada via pip")

os: Biblioteca padrão do Python ou não instalada via pip
openai: 0.28.0
ffmpeg: Biblioteca padrão do Python ou não instalada via pip
librosa: 0.10.2.post1
noisereduce: 3.0.3
soundfile: 0.13.0
pydub: 0.25.1
glob: Biblioteca padrão do Python ou não instalada via pip
